# 🚀 Buildify 3DGS Tutorial

C++23 기반의 3D Gaussian Splatting 라이브러리를 Python에서 사용하는 방법을 배워보세요!

## 📚 목차
1. [기본 설정](#setup)
2. [수학 유틸리티](#math)
3. [엔진 및 씬 관리](#engine)
4. [카메라 시스템](#camera)
5. [엔티티 관리](#entities)
6. [렌더링 시스템](#rendering)

## 1. 기본 설정 {#setup}

먼저 Buildify를 import하고 기본 설정을 해보겠습니다.

In [ ]:
import buildify
import numpy as np
import matplotlib.pyplot as plt

# 로그 레벨 설정
buildify.utils.set_log_level(buildify.utils.LogLevel.Info)

print(f"📦 Buildify 버전: {buildify.__version__}")
print(f"🔧 사용 가능한 모듈: {[attr for attr in dir(buildify) if not attr.startswith('_')]}")

## 2. 수학 유틸리티 {#math}

C++23의 concepts을 사용한 현대적인 수학 라이브러리를 체험해보세요.

In [ ]:
# Vector3 생성 및 연산
v1 = buildify.utils.Vector3(1.0, 2.0, 3.0)
v2 = buildify.utils.Vector3(4.0, 5.0, 6.0)

print(f"벡터 1: {v1}")
print(f"벡터 2: {v2}")
print(f"덧셈: {v1 + v2}")
print(f"스칼라 곱: {v1 * 2.0}")
print(f"내적: {v1.dot(v2)}")
print(f"외적: {v1.cross(v2)}")
print(f"길이: {v1.length():.3f}")
print(f"정규화: {v1.normalized()}")

In [ ]:
# 쿼터니언 회전
import math

# Y축 중심으로 45도 회전
axis = buildify.utils.Vector3(0, 1, 0)
angle = math.pi / 4  # 45도
rotation = buildify.utils.Quaternion.from_axis_angle(axis, angle)

print(f"회전 쿼터니언: ({rotation.x:.3f}, {rotation.y:.3f}, {rotation.z:.3f}, {rotation.w:.3f})")

# 회전 행렬로 변환
rotation_matrix = rotation.to_matrix()
print("회전 행렬 생성 완료")

In [ ]:
# Transform 시스템
transform = buildify.utils.Transform()
transform.position = buildify.utils.Vector3(5, 10, 15)
transform.rotation = rotation
transform.scale = buildify.utils.Vector3(2, 2, 2)

# 최종 변환 행렬 계산
final_matrix = transform.to_matrix()
print("변환 행렬 (위치 + 회전 + 스케일) 생성 완료")
print(f"위치: {transform.position}")
print(f"스케일: {transform.scale}")

## 3. 엔진 및 씬 관리 {#engine}

C++23의 현대적인 메모리 관리와 RAII 패턴을 사용하는 엔진을 체험해보세요.

In [ ]:
# 엔진 생성 및 초기화
engine = buildify.core.Engine()
success = engine.initialize()

print(f"엔진 초기화: {'✅ 성공' if success else '❌ 실패'}")
print(f"엔진 실행 중: {engine.is_running()}")

In [ ]:
# 씬 생성
scene1 = engine.create_scene("MainScene")
scene2 = engine.create_scene("TestScene")

print(f"씬 1: '{scene1.get_name()}'")
print(f"씬 2: '{scene2.get_name()}'")

# 활성 씬 설정
engine.set_active_scene(scene1)
print(f"활성 씬 설정됨: {scene1.get_name()}")

## 4. 카메라 시스템 {#camera}

3D 그래픽스의 핵심인 카메라 시스템을 설정해보세요.

In [ ]:
# 카메라 생성
camera = buildify.core.Camera("MainCamera")

# 원근 투영 설정
fov = 45.0  # 시야각
aspect_ratio = 16.0 / 9.0  # 종횡비
near_plane = 0.1  # 가까운 평면
far_plane = 1000.0  # 먼 평면

camera.set_perspective(fov, aspect_ratio, near_plane, far_plane)
print(f"카메라 투영 설정: FOV={fov}°, 종횡비={aspect_ratio}, 클리핑={near_plane}-{far_plane}")

In [ ]:
# 카메라 위치 및 방향 설정
camera_transform = camera.get_transform()
camera_transform.position = buildify.utils.Vector3(0, 5, 10)
camera.set_transform(camera_transform)

# 원점을 바라보도록 설정
target = buildify.utils.Vector3(0, 0, 0)
up = buildify.utils.Vector3(0, 1, 0)
camera.look_at(target, up)

print(f"카메라 위치: {camera_transform.position}")
print(f"카메라가 {target}을 바라봄")

In [ ]:
# 씬에 카메라 추가 및 활성화
scene1.add_entity(camera)
scene1.set_active_camera(camera)

print("✅ 카메라가 씬에 추가되고 활성화됨")

# 뷰 및 투영 행렬 가져오기
view_matrix = camera.get_view_matrix()
projection_matrix = camera.get_projection_matrix()

print("📐 뷰 행렬과 투영 행렬 계산 완료")

## 5. 엔티티 관리 {#entities}

씬 내의 3D 객체들을 관리하는 엔티티 시스템을 체험해보세요.

In [ ]:
# 여러 엔티티 생성
entities = []

for i in range(5):
    entity = buildify.core.Entity(f"Cube_{i}")
    
    # 랜덤 위치 설정
    import random
    x = random.uniform(-10, 10)
    y = random.uniform(0, 5)
    z = random.uniform(-10, 10)
    
    transform = entity.get_transform()
    transform.position = buildify.utils.Vector3(x, y, z)
    transform.scale = buildify.utils.Vector3(1, 1, 1)
    entity.set_transform(transform)
    
    # 씬에 추가
    scene1.add_entity(entity)
    entities.append(entity)
    
    print(f"엔티티 '{entity.get_name()}' 생성: 위치=({x:.1f}, {y:.1f}, {z:.1f})")

print(f"\n✅ 총 {len(entities)}개의 엔티티가 생성됨")

In [ ]:
# 엔티티 검색
search_name = "Cube_2"
found_entity = scene1.find_entity(search_name)

if found_entity:
    print(f"✅ 엔티티 '{search_name}' 발견!")
    transform = found_entity.get_transform()
    print(f"   위치: {transform.position}")
    print(f"   스케일: {transform.scale}")
else:
    print(f"❌ 엔티티 '{search_name}'를 찾을 수 없음")

## 6. 렌더링 시스템 {#rendering}

OpenGL 기반의 현대적인 렌더링 시스템을 설정해보세요.

In [ ]:
# OpenGL 렌더러 생성
renderer = buildify.core.OpenGLRenderer()

# 렌더 타겟 설정
render_target = buildify.core.RenderTarget()
render_target.width = 1920
render_target.height = 1080
render_target.samples = 4  # 4x MSAA

print(f"🎨 렌더러 생성됨")
print(f"📺 해상도: {render_target.width}x{render_target.height}")
print(f"🔍 안티알리어싱: {render_target.samples}x MSAA")
print("\n⚠️  참고: 실제 렌더링은 OpenGL 컨텍스트가 필요합니다")

In [ ]:
# 엔진에 렌더러 설정
# engine.set_renderer(renderer)  # 주석: OpenGL 컨텍스트 필요

print("🔧 렌더링 파이프라인 설정:")
print("   1. 렌더 타겟 생성")
print("   2. 프레임버퍼 설정")
print("   3. 뷰포트 설정")
print("   4. 깊이 테스트 활성화")
print("   5. 백페이스 컬링 활성화")

## 7. 시뮬레이션 실행

엔진의 업데이트 루프를 실행해보세요.

In [ ]:
import time

# 업데이트 콜백 함수 정의
frame_count = 0
start_time = time.time()

def update_callback(delta_time):
    global frame_count, start_time
    frame_count += 1
    
    if frame_count % 10 == 0:  # 10프레임마다 출력
        elapsed = time.time() - start_time
        fps = frame_count / elapsed
        print(f"📊 프레임 #{frame_count}: FPS={fps:.1f}, dt={delta_time*1000:.1f}ms")

# 콜백 등록
engine.add_update_callback(update_callback)
print("🔄 업데이트 콜백 등록됨")

In [ ]:
# 시뮬레이션 실행 (50프레임)
print("🏃 시뮬레이션 시작...")

for frame in range(50):
    dt = 0.016667  # 60 FPS (1/60초)
    
    # 엔진 업데이트
    engine.update(dt)
    
    # 씬 업데이트
    scene1.update(dt)
    
    # 렌더링 시뮬레이션
    # engine.render()  # 주석: OpenGL 컨텍스트 필요
    
    # 프레임 간격
    time.sleep(0.01)  # 실제로는 VSync에 의해 조절됨

print("✅ 시뮬레이션 완료")

## 8. 정리

리소스를 정리하고 엔진을 종료합니다.

In [ ]:
# 엔진 종료
engine.shutdown()

print("🏁 엔진 종료됨")
print(f"📈 총 처리된 프레임: {frame_count}")
print("\n🎉 Buildify C++23 튜토리얼 완료!")
print("\n다음 단계:")
print("  • 3D Gaussian Splatting 알고리즘 구현")
print("  • Blender 통합 (.blend 파일 로딩)")
print("  • PyTorch 텐서 연동")
print("  • 실시간 렌더링 데모")